<h1 style="text-align: center;"> Tarea Machine Learning </h1>
<h1 style="text-align: right; font-size: 24px; margin-right: 10px;">Guillermo Díaz Aguado</h1>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


# Mostrar todas las filas y columnas en outputs
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)



In [3]:
df = pd.read_excel("datos_tarea25.xlsx")
len(df)

4340

In [4]:
# Quito los "-" de la columna "Levy"
df["Levy"] = df["Levy"].replace("-", np.nan)


def poner_1_y_0(df, columna, str1, str0):
    return df[columna].map({str1:1, str0:0})

# Cambiar las columnas "Leather interior", "Gear box type", "Wheel", "Color" ->  a valores de 0 y 1
df["Leather interior"] = poner_1_y_0(df, "Leather interior", "Yes", "No")
df["Gear box type"] = poner_1_y_0(df, "Gear box type", "Tiptronic", "Automatic")
df["Wheel"] = poner_1_y_0(df, "Wheel", "Left wheel", "Right-hand drive")
df["Color"] = poner_1_y_0(df, "Color", "White", "Black")


# Crear columna de 1 y 0 que indica si tiene turbo
df["Tiene Turbo"] = df["Engine volume"].astype(str).str.contains("Turbo").astype(int)

# Limpiar la columna "Engine volume" y convertir a numérico
df["Engine volume"] = df["Engine volume"].astype(str).str.replace(r"Turbo", "", regex=True).astype(float)

# Limpiar la columna "Mileage" quitando "km"
df["Mileage"] = df["Mileage"].astype(str).str.replace(r"km", "", regex=True).astype(int)



In [5]:
df.dtypes

Price                 int64
Levy                 object
Manufacturer         object
Prod. year            int64
Category             object
Leather interior      int64
Fuel type            object
Engine volume       float64
Mileage               int32
Cylinders             int64
Gear box type         int64
Drive wheels         object
Wheel                 int64
Color                 int64
Airbags               int64
Tiene Turbo           int32
dtype: object

In [6]:
def rango_intercuartilico(series):
    # Calculo el Q1, el Q3 y el IQR
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1

    # Establezco los limites para detectar outliers
    lim_inf = Q1 - 1.5*IQR
    lim_sup = Q3 + 1.5*IQR

    #Detecto los valores atipicos
    outliers = ((series<lim_inf) | (series>lim_sup)).astype(int)
    return outliers

def zscore_outliers(series, umbral=3):
    media = series.mean()
    std = series.std()

    z_scores = (series - media) / std
    return (z_scores.abs() > umbral).astype(int) 

In [10]:
def valores_atipicos(df, show_1=True):
    # Primero seleccionaremos los datos que sean nuemricos
    numericas = list(df.select_dtypes(include=['int', 'int32', 'int64','float', 'float32', 'float64']))
    
    numericas = [x for x in numericas if x not in ["Leather interior", "Gear box type", "Wheel", "Color", "Tiene Turbo"]]

    atipicos_por_columna = []

    for columna in numericas:
        atipicos = pd.DataFrame({
            "Intercuartilico":rango_intercuartilico(df[columna]),
            "Z-Score": zscore_outliers(df[columna])
        })
        atipicos_por_columna.append(atipicos)

    final = pd.MultiIndex.from_product([numericas, atipicos_por_columna[0].columns])

    df_final = pd.concat(atipicos_por_columna, axis=1)
    df_final.columns = final

    if show_1:
        suma = df_final.sum(axis=1)
        return df_final[suma>0]

len(valores_atipicos(df[["Price", "Prod. year", "Mileage", "Cylinders"]]))

389